In [1]:
.libPaths()
library(biomaRt)
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

[1] "/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library"

In [2]:

gene_ids <- unique(read.table("/disk1/wenqing/tmp_data/PFC_s2/filter_snv_for_new_res/GW26_1_1/GABAergic_neurons/detailed_info_genes_with_all_res.txt")[,4])
head(gene_ids)


[1] "RP5-857K21.4" "hsa-mir-6723" "C1orf159"     "CPSF3L"       "MRPL20"      
[6] "ATAD3B"

In [4]:
attributes <- c("ensembl_gene_id_version","ensembl_gene_id", "external_gene_name", "description", "chromosome_name", "start_position", "end_position","gene_biotype")
results <- getBM(attributes = attributes, filters = "external_gene_name", values = gene_ids, mart = ensembl)
results$chromosome_name <- paste0("chr",results$chromosome_name)
head(results)

,ensembl_gene_id_version,ensembl_gene_id,external_gene_name,description,chromosome_name,start_position,end_position,gene_biotype
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
1,ENSG00000198763.3,ENSG00000198763,MT-ND2,mitochondrially encoded NADH:ubiquinone oxidoreductase core subunit 2 [Source:HGNC Symbol;Acc:HGNC:7456],chrMT,4470,5511,protein_coding
2,ENSG00000198804.2,ENSG00000198804,MT-CO1,mitochondrially encoded cytochrome c oxidase I [Source:HGNC Symbol;Acc:HGNC:7419],chrMT,5904,7445,protein_coding
3,ENSG00000198886.2,ENSG00000198886,MT-ND4,mitochondrially encoded NADH:ubiquinone oxidoreductase core subunit 4 [Source:HGNC Symbol;Acc:HGNC:7459],chrMT,10760,12137,protein_coding
4,ENSG00000291375.1,ENSG00000291375,FBXO42,F-box protein 42 [Source:HGNC Symbol;Acc:HGNC:29249],chrHG1343_HG173_HG459_PATCH,50977,156623,protein_coding
5,ENSG00000292225.2,ENSG00000292225,NBPF1,NBPF member 1 [Source:HGNC Symbol;Acc:HGNC:26088],chrHG1343_HG173_HG459_PATCH,1198562,1260797,protein_coding
6,ENSG00000291432.1,ENSG00000291432,KRTAP5-AS1,KRTAP5-1/KRTAP5-2 antisense RNA 1 [Source:HGNC Symbol;Acc:HGNC:27877],chrHG152_PATCH,311329,339160,lncRNA


In [ ]:
length(results$external_gene_name)
length(unique(results$external_gene_name))
length(paste0(results$external_gene_name,"_",results$gene_biotype))
length(unique(paste0(results$external_gene_name,"_",results$gene_biotype)))
results[order(results$external_gene_name),]



In [3]:
##在每个富集到res的基因后面备注该基因的类型
info <- read.table("/disk1/wenqing/tmp_data/PFC_s2/filter_snv_for_new_res/GW26_1_1/GABAergic_neurons/detailed_info_genes_with_all_res.txt")
head(info)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<int>,<int>,<int>
1,chr1,564389,567994,RP5-857K21.4,;,chr1,567988,567989,AC,19,-,438,17949,1
2,chr1,564389,567994,RP5-857K21.4,;,chr1,567989,567990,AC,16,-,319,17946,1
3,chr1,564389,567994,RP5-857K21.4,;,chr1,567993,567994,AC,11,-,423,17873,1
4,chr1,568065,621095,RP5-857K21.4,;,chr1,583220,583221,AG,12,-,22,5122,1
5,chr1,568065,621095,RP5-857K21.4,;,chr1,583222,583223,AG,21,-,30,4846,1
6,chr1,622034,659930,RP5-857K21.4,;,chr1,657326,657327,CA,397,-,825,926,1


In [6]:
###注意：一个基因可能有多个基因id，对应不同的类型，在这个部分的分析中，我们去除了有多重gene biotype的gene
#仅保留1:1的gene
all_gene_types <- c()
for(i in 1:nrow(info)){
    res <- unique(results$gene_biotype[results$external_gene_name==info$V4[i]])
    if(length(res)==1){
        gene_type <- res
    }else{
        gene_type <- 'unKnown'
    }
    all_gene_types <- c(all_gene_types,gene_type)
}

In [7]:
info$V9 <- all_gene_types
table(all_gene_types)

all_gene_types
                            lncRNA                              miRNA 
                               601                                  3 
                          misc_RNA               processed_pseudogene 
                                 2                                  2 
                    protein_coding                              scRNA 
                             19428                                 12 
                            snoRNA   transcribed_processed_pseudogene 
                                 4                                  3 
    transcribed_unitary_pseudogene transcribed_unprocessed_pseudogene 
                                45                                356 
                           unKnown             unprocessed_pseudogene 
                              4995                                139 

In [10]:
write.table(info,"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/detailed_info_genes_with_all_edited_res_with_geneType.txt",quote=F,sep='\t',col.names=F,row.names=F)

In [8]:
##统计各类型基因intron富集的res数量
table(info$V9)


                            lncRNA                              miRNA 
                               601                                  3 
                          misc_RNA               processed_pseudogene 
                                 2                                  2 
                    protein_coding                              scRNA 
                             19428                                 12 
                            snoRNA   transcribed_processed_pseudogene 
                                 4                                  3 
    transcribed_unitary_pseudogene transcribed_unprocessed_pseudogene 
                                45                                356 
                           unKnown             unprocessed_pseudogene 
                              4995                                139 

In [9]:
lncRNA_dat <- info[which(info$V9=='lncRNA'),]
length(unique(paste0(lncRNA_dat$V6,"_",lncRNA_dat$V8)))
mRNA_dat <- info[which(info$V9=='protein_coding'),]
length(unique(paste0(mRNA_dat$V6,"_",mRNA_dat$V8)))

[1] 582

[1] 18281

In [4]:
############
setwd("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/lncRNA_mRNA/3_groups_res/GABA/down")

RNA_res <- read.table("mRNA_exon_down.res")
RNA_res$V9 <- paste0(RNA_res$V1,"_",RNA_res$V2,"_",RNA_res$V3)
head(RNA_res)

,V1,V2,V3,V4,V5,V6,V7,V8,V9
,<chr>,<int>,<int>,<dbl>,<chr>,<int>,<int>,<int>,<chr>
1,chr1,145510938,145514420,128.2240092,chr1,145511928,145511929,1,chr1_145510938_145514420
2,chr1,145510938,145514420,128.2240092,chr1,145511982,145511983,1,chr1_145510938_145514420
3,chr1,145510938,145514420,128.2240092,chr1,145511997,145511998,1,chr1_145510938_145514420
4,chr2,80136923,80532229,0.3557139,chr2,80426140,80426141,1,chr2_80136923_80532229
5,chr2,80136923,80532229,0.3557139,chr2,80426185,80426186,1,chr2_80136923_80532229
6,chr2,80136923,80532229,0.3557139,chr2,80428532,80428533,1,chr2_80136923_80532229


In [ ]:
find_gene_res_num <- function(x){
    gene <- strsplit(uniq_gene_region[x],"_")[[1]][4]
    res_num <- sum(dat$V8[dat$V4==gene])
    return(res_num)
}

In [ ]:
###只计算了ag type的res的基因
#samples=c('GW08','GW12','GW16_1_3','GW16_1_4','GW16_1_9','GW19_1_1','GW19_1_2','GW19_1_3','GW23_1_1','GW23_1_2','GW23_1_3','GW26_1_1','GW26_1_2','GW26_1_3','GW26_1_4','GW26_1_5','GW26_1_6','GW26_1_7','GW26_1_8','GW26_1_9','GW26_1_10')
samples=c('GW23_1_2')
for(one in samples){
    print(one)
    #one='GW08'
    #data_all <- read.table(paste0("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/AEI/GABAergic_neurons/",one,"/geneWithRegular_res.txt"))
    dat <- read.table(paste0("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/AEI/GABAergic_neurons/",one,"/geneWithRegular_res.txt"))

    #dat <- data_all[which(data_all[,7]=='AG'),]

    dat$V10 <- paste0(dat$V1,"_",dat$V2,"_",dat$V3,"_",dat$V4)
    #head(dat)
    uniq_gene_region <- unique(dat$V10)
    #length(uniq_gene_region)

    gene_res_num <- apply(as.matrix(1:length(uniq_gene_region)),1,find_gene_res_num)


    new_dat_col1 <- apply(as.matrix(1:length(uniq_gene_region)),1,function(x) {col1 <- strsplit(uniq_gene_region[x],"_")[[1]][1]})
    new_dat_col2 <- apply(as.matrix(1:length(uniq_gene_region)),1,function(x) {col1 <- strsplit(uniq_gene_region[x],"_")[[1]][2]})
    new_dat_col3 <- apply(as.matrix(1:length(uniq_gene_region)),1,function(x) {col1 <- strsplit(uniq_gene_region[x],"_")[[1]][3]})
    new_dat_col4 <- apply(as.matrix(1:length(uniq_gene_region)),1,function(x) {col1 <- strsplit(uniq_gene_region[x],"_")[[1]][4]})
    new_dat_col5 <- gene_res_num
    new_dat <- as.data.frame(cbind(new_dat_col1,new_dat_col2,new_dat_col3,new_dat_col4,new_dat_col5))
    write.table(new_dat,paste0("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/AEI/GABAergic_neurons/",one,"/geneWzRes_ad.txt"),quote=F,col.names=F,row.names=F,sep='\t')
}

[1] "GW23_1_2"
